# Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import matthews_corrcoef,recall_score, make_scorer

from xgboost import XGBClassifier

# Leitura dos Dados

In [2]:
with open('Arquivos/entradas.npy', 'rb') as f:
    entradas = np.load(f)
with open('Arquivos/saidas.npy', 'rb') as f:
    saidas = np.load(f)

# Testando Modelos

In [3]:
model = XGBClassifier(objective ='binary:logistic', eval_metric='logloss', random_state=42)

dic = {}
x = entradas
y = saidas
        
k_fold_validation = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

acc = cross_val_score(model, x, y.ravel(), scoring='accuracy',  cv=k_fold_validation, n_jobs=-1)
dic['ACC'] = [acc.mean(), acc.std()]

pre = cross_val_score(model, x, y.ravel(), scoring='precision', cv=k_fold_validation, n_jobs=-1)
dic['PRE'] = [pre.mean(), pre.std()]

rec = cross_val_score(model, x, y.ravel(), scoring='recall',    cv=k_fold_validation, n_jobs=-1)
dic['REC'] = [rec.mean(), rec.std()]

f1  = cross_val_score(model, x, y.ravel(), scoring='f1',        cv=k_fold_validation, n_jobs=-1)
dic['F1'] = [f1.mean(),   f1.std()]

auc = cross_val_score(model, x, y.ravel(), scoring='roc_auc',   cv=k_fold_validation, n_jobs=-1)
dic['AUC'] = [auc.mean(), auc.std()]

specificity = make_scorer(recall_score, pos_label=0)
spe = cross_val_score(model, x, y.ravel(), scoring = specificity, cv=k_fold_validation, n_jobs=-1)
dic['SPE'] = [spe.mean(), spe.std()]

MCC_scorer = make_scorer(matthews_corrcoef)
mcc = cross_val_score(model, x, y.ravel(), scoring = MCC_scorer, cv=k_fold_validation, n_jobs=-1)
dic['MCC'] = [mcc.mean(), mcc.std()]

model.fit(x, y.ravel())

metricas = pd.DataFrame.from_dict(dic, orient='index', columns=['M', 'S'])
metricas = metricas.reset_index(level=0)
metricas.columns = ['XGB','Mean', 'STD']

display(metricas)

,XGB,Mean,STD
0,ACC,0.953922,0.005980
1,PRE,0.953203,0.011998
2,REC,0.939207,0.014261
3,F1,0.946023,0.007124
4,AUC,0.990897,0.002619
5,SPE,0.965028,0.009605
6,MCC,0.906095,0.012226


# Usando o modelo pra classificar novas entradas
## Dados Novos 

In [4]:
CLASS_FETO     =    2     #  0-Dado Faltante; 1-Pré-Termo(< 37s); 2-A Termo(>= 37 e < 42s); 3-Pós-Termo(> 42s) 
COMPRIMENT     =   45.4   #  Comprimento em centímetros (cm)  
EXAM_LABZIKA   =    0     #  0-Critério Ausente; 1-Critério Presente : Laboratorial(Zika)
EXAM_LABDEN    =    0     #  0-Critério Ausente; 1-Critério Presente : Laboratorial(Dengue)   
EXAM_LABCHIK   =    0     #  0-Critério Ausente; 1-Critério Presente : Laboratorial(Chikungunya)   
EXAM_LABSTORCH =    0     #  0-Critério Ausente; 1-Critério Presente : Laboratorial(STORCH)    
EXAM_LABOUTROS =    0     #  0-Critério Ausente; 1-Critério Presente : Laboratorial(outros)  
EXAM_IMAGEM    =    0     #  0-Critério Ausente; 1-Critério Presente : Imagem(Ultrassom/Tomografia/Ressonância)  
CLINICO_EPIDEM =    0     #  0-Critério Ausente; 1-Critério Presente : Clínico-Epidemiológico   
DEF_AUDIT      =    2     #  1-Sim; 2-Não
DEF_NEURO      =    2     #  1-Sim; 2-Não
DEF_VISUAL     =    2     #  1-Sim; 2-Não
EXANT_GES      =    1     #  1-Sim(1ºtri); 2-Sim(2º tri); 3-Sim(3º tri); 4-Sim(não lembra a data); 5-Não; 6-Não sabe
FEBRE_GES      =    1     #  1-Sim; 3-Ignorado; 5-Não
GEST_DIAG      =    0     #  0-Dado Faltante; 1-(1ºtri); 2-(2º tri); 3-(3º tri); 4-(4º tri)
GRAVIDEZ       =    1     #  1-Única; 2-Dupla; 3-Tripla ou mais; 4-Ignorado
MICROCEFAL     =    6     #  1-Microcefalia com alteração do SNC; 2- Microcefalia com outras alterações congênitas; 
               # -------- >>> 4-Alterações congênitas sem microcefalia; 6-Microcefalia apenas; 8-Ignorado
NEUROLOG       =    2     #  1-Sim; 2-Não
PERIMCEFAL     =   30.8   #  Perímetro cefálico em centímetros (cm) 
PESO           = 2551.5   #  Peso em gramas (g) 
PRURIDO        =    2     #  1-Sim; 2-Não
RACACOR        =    6     #  2-Negra; 3-Amarela; 4-Parda; 5-Indígena; 6-Ignorado; 7-Branca
SEXO           =    1     #  1-Masculino; 3-Ignorado; 5-Feminino
TPDETECCAO     =    6     #  1-Intrauterino(na gestação); 3-Não detectada; 4-Ignorado; 6-Pós-Parto


entrada_nova = np.array([[CLASS_FETO, COMPRIMENT, EXAM_LABZIKA, EXAM_LABDEN, EXAM_LABCHIK, EXAM_LABSTORCH, EXAM_LABOUTROS,
                          EXAM_IMAGEM, CLINICO_EPIDEM, DEF_AUDIT, DEF_NEURO, DEF_VISUAL, EXANT_GES , FEBRE_GES, GEST_DIAG, 
                          GRAVIDEZ, MICROCEFAL, NEUROLOG, PERIMCEFAL, PESO, PRURIDO, RACACOR, SEXO, TPDETECCAO]])

## Classificando

In [5]:
classe = model.predict(entrada_nova)

if classe[0]==0:
    print('Descartado')
elif classe[0]==1:
    print('Confirmado')

Confirmado
